In [1]:
# -*- coding: utf-8 -*-
from io import open
import os.path
from os import path
import random
import numpy as np
import pickle
import pandas as pd
import scipy.signal
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as pl
import matplotlib.ticker as ticker
from torch import nn
torch.cuda.set_device(0)

from data_prepare_for_many import *
#from data_prepare_nobehavior import *

torch.manual_seed(1)

MAX_LENGTH = 100
torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('ok')
length = 31
predict_length = 30
batch_size = 1
file_path_train = np.array(glob('data/data_interactive/train/*'))
file_path_test = np.array(glob('data/data_interactive/test/*'))
file_path_valid = np.array(glob('data/data_interactive/valid/*'))
#Training_generator1, Test, Valid, WholeSet= get_dataloader(batch_size,length,predict_length)
Training_generator,WholeSet_train= get_dataloader(batch_size,length,predict_length,file_path_train,'train')
Test,WholeSet= get_dataloader(batch_size,length,predict_length,file_path_test,'test')
Valid, WholeSet_valid = get_dataloader(batch_size,length,predict_length,file_path_valid,'valid')

ok
****************************************************************************************************
训练轨迹轨迹条数： 158
---轨迹输入数据结构： torch.Size([1, 31, 44]) ---轨迹输出数据结构： torch.Size([1, 31, 4])
---轨迹长度： 31 ---预测轨迹长度： 30
****************************************************************************************************
测试轨迹轨迹条数： 158
---轨迹输入数据结构： torch.Size([1, 31, 44]) ---轨迹输出数据结构： torch.Size([1, 31, 4])
---轨迹长度： 31 ---预测轨迹长度： 30
****************************************************************************************************
验证轨迹轨迹条数： 158
---轨迹输入数据结构： torch.Size([1, 31, 44]) ---轨迹输出数据结构： torch.Size([1, 31, 4])
---轨迹长度： 31 ---预测轨迹长度： 30


In [2]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(LSTM, self).__init__()
        
        self.hidden_dim=hidden_dim
        self.n_layers = n_layers
        # define an RNN with specified parameters
        # batch_first means that the first dim of the input and output will be the batch_size
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers,dropout=0., batch_first=True,bidirectional=False)
        
        # last, fully-connected layer
        self.fc1 = nn.Linear(hidden_dim,hidden_dim*2)
        self.fc = nn.Linear(hidden_dim*2, output_size)
        # self.fc2 = nn.Linear(X_train.shape[2],output_size)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x, hidden=None):
        batch_size = x.size(0)
        
        # get RNN outputs
        r_out, hidden = self.lstm(x, hidden)
        r_out = r_out.contiguous().view(-1, self.hidden_dim)  
        
        # get final output 
        output = self.fc1(r_out)
        output = self.fc(output)
        output = output.view(batch_size,-1,6) #------num
        output = output[:,-1]
        
        return output
    
if torch.cuda.is_available():
    train_on_gpu = True
else:
    train_on_gpu = False

In [8]:
def trajectory_to_behavior_data(x):
    x = x.to(device)
    std = WholeSet_train.std.repeat(x.shape[0],x.shape[1],1)
    std = std.to(device)
    mn = WholeSet_train.mn.repeat(x.shape[0],x.shape[1],1)
    mn = mn.to(device)
    rg = WholeSet_train.range.repeat(x.shape[0],x.shape[1],1)
    rg = rg.to(device)
    input_return = (x*(rg*std)+mn).detach().cpu()
    #print(std)
    #input_return = x.detach().cpu()
    inputs = torch.from_numpy(input_return[:,:,:-1].detach().cpu().numpy().astype(np.float32))
    inputs = inputs.view(-1,inputs.shape[2])
    y = torch.from_numpy(input_return[:,:,-1].detach().cpu().numpy().astype(np.int))
    y = y.view(-1,1).squeeze()
    return inputs,y

In [11]:
def train(net,epochs,train_loader,valid_loader,clip,lr = 0.0002):
    # train for some number of epochs
    # loss and optimization functions
    loss_min = np.inf
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    counter = 0
    losses_train = []
    losses_valid = []
    accuracies_e = []
    for e in range(epochs):
        # initialize hidden state
        net.train()
        # batch loop
        train_loss = []
        for inputs, labels in train_loader:
            inputs,labels = trajectory_to_behavior_data(inputs)
#             print(inputs.shape,labels.shape,labels)
            inputs = inputs.unsqueeze(2)
            if (train_on_gpu):
                inputs, labels = torch.from_numpy(inputs.numpy().astype(np.float32)).to(device), labels.long().to(device)
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            # zero accumulated gradients
            net.zero_grad()
            # get the output from the model
            output = net(inputs)
            # calculate the loss and perform backprop
            # print(output.shape,labels.shape)
            # print(output[:1])
            loss = criterion(output, labels)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
            train_loss.append(loss.item())
            # loss stats
        # Get validation loss
        val_losses = []
        net.eval()
        accuracies = []
        for inputs, labels in valid_loader:
            print(inputs)
            inputs,labels = trajectory_to_behavior_data(inputs)
            inputs = inputs.unsqueeze(2)
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            # val_h = tuple([each.data for each in val_h])
            if (train_on_gpu):
                inputs, labels = torch.from_numpy(inputs.numpy().astype(np.float32)).to(device), labels.long().to(device)
            #labels = labels.long()
            
            output= net(inputs)
            val_loss = criterion(output, labels)
            _, class_ = torch.max(output, dim=1)
            equal = class_ == labels.view(class_.shape)
            accuracy = torch.mean(equal.type(torch.FloatTensor)).item()
            val_losses.append(val_loss.item())
            accuracies.append(accuracy)               
        net.train()
        losses_train.append(np.mean(train_loss))
        losses_valid.append(np.mean(val_losses))
        accuracies_e.append(np.mean(np.mean(accuracies)))
        print("Epoch: {}/{}...".format(e + 1, epochs),
              "Loss: {}...".format(np.mean(train_loss)),
              "Val Loss: {}...".format(np.mean(val_losses)),
              "val accuracy:{}.".format(np.mean(accuracies))
                     )
        if np.mean(val_losses) < loss_min:
            print('Val loss decreased...')
            torch.save(net.state_dict(),'model/behavior_prediction_0707_43_30.pth')
            loss_min = np.mean(val_losses)
    print('min loss',loss_min)
    plt.plot(losses_train,color='r',label='train_loss')
    plt.plot(losses_valid,color='g',label='valid_loss')
    plt.title('Loss_Trend')
    plt.xlabel('Epoches')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('image/loss_lstm_behavior_0707_43_30.svg',dpi=600)
    plt.savefig('image/loss_lstm_behavior_0707_43_30.png',dpi=600)
    return accuracies_e

def test(net,test_loader):
    # Get test data loss and accuracy
    lr = 0.001

    criterion = nn.CrossEntropyLoss()
    test_losses = []  # track loss
    accuracies = []
    net.eval()
    # iterate over test data
    class_correct = np.zeros(6) #------num
    class_total = np.zeros(6)
    classes = ["Straight", "LeftTurn", "RightTurn", "UTurn", "LeftChange", "RightChange"]
    for inputs, y in test_loader:
        inputs,y = trajectory_to_behavior_data(inputs)
        inputs = inputs.unsqueeze(2)
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        if (train_on_gpu):
            inputs, y = torch.from_numpy(inputs.numpy().astype(np.float32)).to(device), y.long().to(device)
        # get predicted outputs
       # inputs,y = torch.from_numpy(inputs.numpy().astype(np.float32)),y.long()
        output= net(inputs)
        
        # calculate loss
        test_loss = criterion(output, y)
        _, class_ = torch.max(output, dim=1)
        equal = class_ == y.view(class_.shape)
        for i in range(y.shape[0]):
            label = y.data[i].item()
            class_correct[label] += equal[i].item()
            class_total[label] += 1
        accuracy = torch.mea128n(equal.type(torch.FloatTensor)).item()
        test_losses.append(test_loss.item())
        accuracies.append(accuracy)

    # -- stats! -- ##
    # avg test loss
    print('Test Loss: {:.20f}\n'.format(test_loss.item()))
    for i in range(4):
        if class_total[i]>0:
            print('Test Accuracy of {}:{:.4f}({}/{})'.format(classes[i],100*class_correct[i]/class_total[i],
                 int(np.sum(class_correct[i])),
                 int(np.sum(class_total[i]))))
        else:
            print('Test Accuracy of {}:N/A(no examples)'.format(classes[i]))
    print('Test Accuracy(Overall):{:.4f} ({}/{})'.format(100*np.sum(class_correct)/np.sum(class_total),
                                                    int(np.sum(class_correct)),
                                                    int(np.sum(class_total))))
    print("Test loss: {:.10f}".format(np.mean(test_losses)),'Test Accuracy:{}'.format(np.mean(accuracies)))

In [10]:
net = LSTM(1,6,256,2) #------num
from time import time
if train_on_gpu:
    net.to(device)
start = time()
epochs = 100
accuracy = train(net,epochs,Training_generator,Valid,clip=5,lr=0.0001)
print('Training time is:',time()-start,'s')

tensor([[[-1.9866,  0.9429,  2.8758,  ...,     nan,     nan,  2.9091],
         [-1.8810,  0.9270,  2.8142,  ...,     nan,     nan,  2.9091],
         [-1.7869,  0.9105,  2.7549,  ...,     nan,     nan,  2.9091],
         ...,
         [-1.3915,  0.4810,  1.3847,  ...,     nan,     nan, -1.0069],
         [-1.3758,  0.4773,  1.3351,  ...,     nan,     nan, -1.0069],
         [-1.3608,  0.4926,  1.2860,  ...,     nan,     nan, -1.0069]]],
       dtype=torch.float64)
tensor([[[ 0.9159, -2.0042, -0.6845,  ...,     nan,     nan, -1.0069],
         [ 0.9202, -1.9704, -0.6835,  ...,     nan,     nan, -1.0069],
         [ 0.9221, -1.9376, -0.6825,  ...,     nan,     nan, -1.0069],
         ...,
         [ 0.5667, -1.3285, -0.7525,  ...,     nan,     nan,  0.5595],
         [ 0.5464, -1.3237, -0.7594,  ...,     nan,     nan,  0.5595],
         [ 0.5274, -1.3134, -0.7669,  ...,     nan,     nan,  0.5595]]],
       dtype=torch.float64)
tensor([[[ 1.3443,  0.7621, -1.1300,  ...,     nan,     nan,

tensor([[[-1.0395,  0.2954,  1.4515,  ...,     nan,     nan, -1.0069],
         [-1.0250,  0.2992,  1.4097,  ...,     nan,     nan, -1.0069],
         [-1.0072,  0.3022,  1.3683,  ...,     nan,     nan, -1.0069],
         ...,
         [-0.2344, -0.1378,  0.5592,  ...,     nan,     nan, -0.2237],
         [-0.2515, -0.2188,  0.5347,  ...,     nan,     nan, -0.2237],
         [-0.2733, -0.2989,  0.5098,  ...,     nan,     nan, -0.2237]]],
       dtype=torch.float64)
tensor([[[ 2.3921,  0.4239, -1.2852,  ...,     nan,     nan, -1.0069],
         [ 2.3915,  0.4247, -1.2519,  ...,     nan,     nan, -1.0069],
         [ 2.3912,  0.4247, -1.2186,  ...,     nan,     nan, -1.0069],
         ...,
         [ 2.4319,  0.4495, -0.3423,  ...,     nan,     nan, -1.0069],
         [ 2.4306,  0.4608, -0.3081,  ...,     nan,     nan, -1.0069],
         [ 2.4303,  0.4672, -0.2740,  ...,     nan,     nan, -1.0069]]],
       dtype=torch.float64)
tensor([[[-1.5747,  0.6307,  1.9525,  ...,     nan,     nan,

tensor([[[ 0.9202, -1.9704, -0.6835,  ...,     nan,     nan, -1.0069],
         [ 0.9221, -1.9376, -0.6825,  ...,     nan,     nan, -1.0069],
         [ 0.9214, -1.9057, -0.6814,  ...,     nan,     nan, -1.0069],
         ...,
         [ 0.5464, -1.3237, -0.7594,  ...,     nan,     nan,  0.5595],
         [ 0.5274, -1.3134, -0.7669,  ...,     nan,     nan,  0.5595],
         [ 0.5136, -1.3165, -0.7746,  ...,     nan,     nan,  0.5595]]],
       dtype=torch.float64)
tensor([[[ 0.6195, -1.4084, -0.7174,  ...,     nan,     nan,  0.5595],
         [ 0.6185, -1.3862, -0.7231,  ...,     nan,     nan,  0.5595],
         [ 0.6230, -1.3701, -0.7285,  ...,     nan,     nan,  0.5595],
         ...,
         [ 0.1626, -0.7844, -1.0129,  ...,     nan,     nan,  0.5595],
         [ 0.1544, -0.7056, -1.0285,  ...,     nan,     nan,  0.5595],
         [ 0.1248, -0.6408, -1.0445,  ...,     nan,     nan,  0.5595]]],
       dtype=torch.float64)
tensor([[[-0.4065,  0.1902,  0.8328,  ...,     nan,     nan,

tensor([[[ 0.7645, -1.6381, -0.6829,  ...,     nan,     nan, -1.0069],
         [ 0.7409, -1.6194, -0.6855,  ...,     nan,     nan, -1.0069],
         [ 0.7276, -1.6076, -0.6887,  ...,     nan,     nan,  0.5595],
         ...,
         [ 0.2941, -1.1929, -0.8766,  ...,     nan,     nan,  0.5595],
         [ 0.2711, -1.1737, -0.8896,  ...,     nan,     nan,  0.5595],
         [ 0.2488, -1.1472, -0.9031,  ...,     nan,     nan,  0.5595]]],
       dtype=torch.float64)
tensor([[[-0.2255, -0.0776,  0.5833,  ...,     nan,     nan, -0.2237],
         [-0.2344, -0.1378,  0.5592,  ...,     nan,     nan, -0.2237],
         [-0.2515, -0.2188,  0.5347,  ...,     nan,     nan, -0.2237],
         ...,
         [ 0.0193, -1.6482, -0.0478,  ...,     nan,     nan, -0.2237],
         [ 0.0321, -1.6954, -0.0664,  ...,     nan,     nan, -0.2237],
         [ 0.0457, -1.7384, -0.0847,  ...,     nan,     nan, -0.2237]]],
       dtype=torch.float64)
tensor([[[ 0.6185, -1.3862, -0.7231,  ...,     nan,     nan,

tensor([[[-0.2255, -0.0776,  0.5833,  ...,     nan,     nan, -0.2237],
         [-0.2344, -0.1378,  0.5592,  ...,     nan,     nan, -0.2237],
         [-0.2515, -0.2188,  0.5347,  ...,     nan,     nan, -0.2237],
         ...,
         [ 0.0193, -1.6482, -0.0478,  ...,     nan,     nan, -0.2237],
         [ 0.0321, -1.6954, -0.0664,  ...,     nan,     nan, -0.2237],
         [ 0.0457, -1.7384, -0.0847,  ...,     nan,     nan, -0.2237]]],
       dtype=torch.float64)
tensor([[[-0.3175,  0.1692,  0.7787,  ...,     nan,     nan, -1.0069],
         [-0.2836,  0.1572,  0.7530,  ...,     nan,     nan, -0.2237],
         [-0.2576,  0.1391,  0.7279,  ...,     nan,     nan, -0.2237],
         ...,
         [-0.0911, -1.2478,  0.1125,  ...,     nan,     nan, -0.2237],
         [-0.0823, -1.3039,  0.0915,  ...,     nan,     nan, -0.2237],
         [-0.0714, -1.3711,  0.0706,  ...,     nan,     nan, -0.2237]]],
       dtype=torch.float64)
tensor([[[ 0.7919, -1.6724, -0.6807,  ...,     nan,     nan,

tensor([[[ 5.9029e-01, -1.3260e+00, -7.4598e-01,  ...,         nan,
                  nan,  5.5949e-01],
         [ 5.6666e-01, -1.3285e+00, -7.5252e-01,  ...,         nan,
                  nan,  5.5949e-01],
         [ 5.4643e-01, -1.3237e+00, -7.5944e-01,  ...,         nan,
                  nan,  5.5949e-01],
         ...,
         [ 5.7777e-05, -4.9880e-01, -1.0979e+00,  ...,         nan,
                  nan,  5.5949e-01],
         [-3.8681e-02, -4.5936e-01, -1.1174e+00,  ...,         nan,
                  nan,  5.5949e-01],
         [-7.9406e-02, -4.1971e-01, -1.1379e+00,  ...,         nan,
                  nan,  5.5949e-01]]], dtype=torch.float64)
tensor([[[ 1.4047,  0.5772, -1.1734,  ...,     nan,     nan, -1.0069],
         [ 1.3880,  0.6168, -1.1619,  ...,     nan,     nan, -1.0069],
         [ 1.3718,  0.6595, -1.1508,  ...,     nan,     nan, -1.0069],
         ...,
         [ 0.8266,  1.5519, -1.0126,  ...,     nan,     nan,  1.3427],
         [ 0.8100,  1.5717, -1.0139

tensor([[[ 0.9159, -2.0042, -0.6845,  ...,     nan,     nan, -1.0069],
         [ 0.9202, -1.9704, -0.6835,  ...,     nan,     nan, -1.0069],
         [ 0.9221, -1.9376, -0.6825,  ...,     nan,     nan, -1.0069],
         ...,
         [ 0.5667, -1.3285, -0.7525,  ...,     nan,     nan,  0.5595],
         [ 0.5464, -1.3237, -0.7594,  ...,     nan,     nan,  0.5595],
         [ 0.5274, -1.3134, -0.7669,  ...,     nan,     nan,  0.5595]]],
       dtype=torch.float64)
tensor([[[ 2.3921,  0.4239, -1.2852,  ...,     nan,     nan, -1.0069],
         [ 2.3915,  0.4247, -1.2519,  ...,     nan,     nan, -1.0069],
         [ 2.3912,  0.4247, -1.2186,  ...,     nan,     nan, -1.0069],
         ...,
         [ 2.4319,  0.4495, -0.3423,  ...,     nan,     nan, -1.0069],
         [ 2.4306,  0.4608, -0.3081,  ...,     nan,     nan, -1.0069],
         [ 2.4303,  0.4672, -0.2740,  ...,     nan,     nan, -1.0069]]],
       dtype=torch.float64)
tensor([[[ 1.4958,  0.4251, -1.2365,  ...,     nan,     nan,

tensor([[[-1.0585,  0.2854,  1.5360,  ...,     nan,     nan, -1.0069],
         [-1.0506,  0.2908,  1.4936,  ...,     nan,     nan, -1.0069],
         [-1.0395,  0.2954,  1.4515,  ...,     nan,     nan, -1.0069],
         ...,
         [-0.2196, -0.0131,  0.6074,  ...,     nan,     nan, -0.2237],
         [-0.2255, -0.0776,  0.5833,  ...,     nan,     nan, -0.2237],
         [-0.2344, -0.1378,  0.5592,  ...,     nan,     nan, -0.2237]]],
       dtype=torch.float64)
tensor([[[ 0.9675,  1.3845, -1.0166,  ...,     nan,     nan,  1.3427],
         [ 0.9486,  1.4114, -1.0146,  ...,     nan,     nan,  1.3427],
         [ 0.9245,  1.4521, -1.0133,  ...,     nan,     nan,  1.3427],
         ...,
         [ 0.2584,  0.9265, -1.1817,  ...,     nan,     nan,  1.3427],
         [ 0.2352,  0.8888, -1.1955,  ...,     nan,     nan,  1.3427],
         [ 0.2161,  0.8729, -1.2097,  ...,     nan,     nan,  1.3427]]],
       dtype=torch.float64)
tensor([[[ 1.3880,  0.6168, -1.1619,  ...,     nan,     nan,

KeyboardInterrupt: 

In [ ]:
plt.plot(accuracy,color='b',label='accuracy')
plt.title('Accuracy_Trend')
plt.xlabel('Epoches')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('image/accuracy_lstm_behavior_25.svg',dpi=300)
plt.savefig('image/accuracy_lstm_behavior_25.png',dpi=300)

In [13]:
net_test = LSTM(1,6,256,2) #------num
net_test.load_state_dict(torch.load('model/behavior_prediction_0707_43_30.pth',map_location='cuda'))
if train_on_gpu:
    net_test.cuda()   
test(net_test,test_loader)

Test Loss: 0.01098422985523939133

Test Accuracy of Straight:99.3561(86407/86967)
Test Accuracy of LeftTurn:97.7555(7317/7485)
Test Accuracy of RightTurn:95.9444(6624/6904)
Test Accuracy of UTurn:96.3964(321/333)
Test Accuracy(Overall):98.5337 (105504/107074)
Test loss: 0.0356363235 Test Accuracy:0.9853546864035502


In [14]:
net_test.cuda().eval()
from time import time
# An example input you would normally provide to your model's forward() method.
#example = torch.rand(1, 4)

example = torch.ones(1,43,1).cuda()
t = time()
out = net_test(example)
print('time',(time()-t)*1000,'ms')
print(out)
# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(net_test, example)
traced_script_module.save("model/behavior_prediction_LSTM_0707_31_30_cuda.pt")
example1 = torch.zeros(1,43,1).cuda()
out= traced_script_module(example)
out1 = traced_script_module(example1)
print(out)
print(out1)

time 2.579212188720703 ms
tensor([[ 12.6786,  -4.3527,  -0.7258, -23.5159,  -4.1703, -43.7727]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 12.6786,  -4.3527,  -0.7258, -23.5159,  -4.1703, -43.7727]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[  0.1717,  -2.1946,   0.9066,  -3.3042,   2.1774, -14.9736]],
       device='cuda:0', grad_fn=<SelectBackward>)
